In [97]:
import pandas as pd
from collections import defaultdict 
import numpy as np
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses","Diagnosis"]
df = pd.read_csv('breast-cancer-wisconsin.data', na_values='?',    
         header=None, index_col=['ID'], names = headers) 
df = df.reset_index(drop=True)
df = df.fillna(0)
df.describe()

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.463519,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.640708,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [98]:
ratio = 0.8
count = int(ratio*df.shape[0])
train_df = df.iloc[0:count,:]
test_df = df.iloc[count:,:]

In [99]:
def isPure(df):
    lastCol = df.iloc[:,-1]
    if(len(np.unique(lastCol)) == 1):
        return True
    return False

In [100]:
def classify(df):
    classes,counts = np.unique(df.values,return_counts=True)
    index = counts.argmax()
    return classes[index]

In [101]:
def splitData(df,col,value):
    dataLeft = df[df[col] <= value]
    dataRight = df[df[col]>value]
    return dataLeft,dataRight

In [102]:
def getPotentialSplits(df):
    ans = defaultdict(list) # ans contains colName -> split values
    for i in range(len(df.columns)-1):
        currColValues = np.unique(df.iloc[:,i].values) # returns sorted unique values
        if len(currColValues) == 1:
            ans[df.columns[i]].append(currColValues[0])
        else:
            for j in range(1,len(currColValues)):
                ans[df.columns[i]].append((currColValues[j-1]+currColValues[j])/2)
    return ans

In [103]:
def calcProb(df):
    labels = df.iloc[:,-1]
    cols,counts = np.unique(labels,return_counts=True)
    if len(counts)>1:
        return counts[0]/(counts[0]+counts[1])
    else:
        return 1

In [104]:
def totalError(leftDf,rightDf,metric):
    probLeft = calcProb(leftDf)
    probRight = calcProb(rightDf)
    if(metric == "gini"):
        leftMetric = 2*probLeft*(1-probLeft)
        rightMetric = 2*probRight*(1-probRight)
    elif(metric == 'entropy'):
        if(probLeft == 0 or probLeft == 1):
            leftMetric = 0
        else:
            leftMetric = (-1)*((probLeft*np.log2(probLeft))+((1-probLeft)*np.log2(1-probLeft)))
        if(probRight == 0 or probRight == 1):
            rightMetric = 0
        else:
            rightMetric = (-1)*((probRight*np.log2(probRight))+((1-probRight)*np.log2(1-probRight)))
    else:
        leftMetric = min(probLeft,1-probLeft)
        rightMetric = min(probRight,1-probRight)
    totalCount = leftDf.shape[0] + rightDf.shape[0]
    return (leftMetric*(leftDf.shape[0]/totalCount)) + (rightMetric*(rightDf.shape[0]/totalCount))

In [105]:
def getBestSplit(df,metric):
    #print("len of cols in getBestSplit : ", len(df.columns))
    potentialSplits = getPotentialSplits(df)
    #print(potentialSplits)
    Error = 100000 # arbitrary max value
    for i in range(len(df.columns)-1): #want to exclude the last col(labels)
        for j in range(len(potentialSplits[df.columns[i]])):
            leftDf,rightDf = splitData(df,df.columns[i],potentialSplits[df.columns[i]][j])
            currError = totalError(leftDf,rightDf,metric)
            #print("error is : ", currError)
            if(currError <= Error):
                Error = currError
                ans_col = df.columns[i]
                ans_val = potentialSplits[df.columns[i]][j]
    return ans_col,ans_val

In [106]:
class treeNode:
    def __init__(self,df,metric):
        self.df = df # last colum has all the labels
        self.isLeaf = False
        if metric not in ['misclassification_rate', 'entropy', 'gini']:
            print("Error metric not defined : ",metric)
            return None
        self.metric = metric
        self.final_class = classify(df)
        if (isPure(df)):
            self.isLeaf = True
            self.final_class = df.iloc[:,-1].values[0] # class contains the final classification
        elif(len(self.df.columns) == 1):
            self.isLeaf = True
            self.final_class = classify(df)
        else:
            self.colToSplitOn,self.valueToSplitBy = getBestSplit(df,self.metric)
            #print("col to split : ",self.colToSplitOn," value is : ",self.valueToSplitBy)
            dataLeft,dataRight = splitData(self.df,self.colToSplitOn,self.valueToSplitBy)
            if dataLeft.shape[0] > 0:
                self.left = treeNode(dataLeft.drop(columns=[self.colToSplitOn]),metric)
            if dataRight.shape[0] > 0:
                self.right = treeNode(dataRight.drop(columns=[self.colToSplitOn]),metric)
        

In [107]:
def predictHelper(dataRow,treeNode):
    if treeNode is None:
        print("Error treeNode shouldn't be None. Train first ")
    if treeNode.isLeaf:
        return treeNode.final_class
    else:
        col = treeNode.colToSplitOn
        val = treeNode.valueToSplitBy
        if not hasattr(treeNode,'left'):
            return predictHelper(dataRow,treeNode.right)
        if not hasattr(treeNode,'right'):
            return predictHelper(dataRow,treeNode.left)
        if dataRow[col]<= val and hasattr(treeNode,'left'):
            return predictHelper(dataRow,treeNode.left)
        elif dataRow[col] > val and hasattr(treeNode,'right'):
            return predictHelper(dataRow,treeNode.right)
        

In [108]:
def predict(df,treeNode):
    ans = []
    for i in range(df.shape[0]):
        ans.append(predictHelper(df.iloc[i],treeNode))
        #print("i  is: ",i,"ans is ", ans)
    return ans

In [144]:
def predict_maxDepth_Helper(dataRow,treeNode,depth,currDepth):
    if treeNode is None:
        print("Error treeNode shouldn't be None. Train first ")
    if treeNode.isLeaf:
        return treeNode.final_class
    if depth == currDepth:
        return treeNode.final_class
    else:
        col = treeNode.colToSplitOn
        val = treeNode.valueToSplitBy
        if not hasattr(treeNode,'left'):
            return predict_maxDepth_Helper(dataRow,treeNode.right,depth,currDepth+1)
        if not hasattr(treeNode,'right'):
            return predict_maxDepth_Helper(dataRow,treeNode.left,depth,currDepth+1)
        if dataRow[col]<= val and hasattr(treeNode,'left'):
            return predict_maxDepth_Helper(dataRow,treeNode.left,depth,currDepth+1)
        elif dataRow[col] > val and hasattr(treeNode,'right'):
            return predict_maxDepth_Helper(dataRow,treeNode.right,depth,currDepth+1)

In [145]:
def predict_maxDepth(df,treeNode,depth):
    ans = []
    for i in range(df.shape[0]):
        ans.append(predict_maxDepth_Helper(df.iloc[i],treeNode,depth,0))
        #print("i  is: ",i,"ans is ", ans)
    return ans

In [109]:
tree_gini = treeNode(train_df,'gini')
tree_entropy = treeNode(train_df,'entropy')
tree_misclassification_rate = treeNode(train_df,'misclassification_rate')

In [110]:
df_input = test_df.iloc[:,:-1]

In [111]:
def getAccuracy(y_test_pred,test_df):
    num_correct = np.sum(y_test_pred == test_df.iloc[:,-1])
    num_test = test_df.shape[0]
    accuracy = float(num_correct) / num_test
    return num_correct,num_test,accuracy

In [112]:
y_test_pred_gini = predict(df_input,tree_gini)
y_test_pred_entropy = predict(df_input,tree_entropy)
y_test_pred_mis = predict(df_input,tree_misclassification_rate)

#print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))
print('For gini Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_gini,test_df))
print('For entropy Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_entropy,test_df))
print('For misclassification rate Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_mis,test_df))

For gini Got 131 / 140 correct => accuracy: 0.935714
For entropy Got 136 / 140 correct => accuracy: 0.971429
For misclassification rate Got 137 / 140 correct => accuracy: 0.978571


 - Standardisation and normalization does not have any effect because we are not calculating distance in any sense. 
 - Just comparing the values in a range.

In [155]:
y_test_pred_gini_3 = predict_maxDepth(df_input,tree_gini,3)
y_test_pred_entropy_3 = predict_maxDepth(df_input,tree_entropy,3)
y_test_pred_mis_3 = predict_maxDepth(df_input,tree_misclassification_rate,3)

y_test_pred_gini_5 = predict_maxDepth(df_input,tree_gini,5)
y_test_pred_entropy_5 = predict_maxDepth(df_input,tree_entropy,5)
y_test_pred_mis_5 = predict_maxDepth(df_input,tree_misclassification_rate,5)


y_test_pred_gini_7 = predict_maxDepth(df_input,tree_gini,7)
y_test_pred_entropy_7 = predict_maxDepth(df_input,tree_entropy,7)
y_test_pred_mis_7 = predict_maxDepth(df_input,tree_misclassification_rate,7)

print('with 3 levels gini Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_gini_3,test_df))
print('with 3 levels entropy Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_entropy_3,test_df))
print('with 3 levels misclassification rate Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_mis_3,test_df))
print()
print('with 5 levels gini Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_gini_5,test_df))
print('with 5 levels entropy Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_entropy_5,test_df))
print('with 5 levels misclassification rate Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_mis_5,test_df))
print()
print('with 7 levels gini Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_gini_7,test_df))
print('with 7 levels entropy Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_entropy_7,test_df))
print('with 7 levels misclassification rate Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_mis_7,test_df))

with 3 levels gini Got 32 / 140 correct => accuracy: 0.228571
with 3 levels entropy Got 126 / 140 correct => accuracy: 0.900000
with 3 levels misclassification rate Got 37 / 140 correct => accuracy: 0.264286

with 5 levels gini Got 126 / 140 correct => accuracy: 0.900000
with 5 levels entropy Got 135 / 140 correct => accuracy: 0.964286
with 5 levels misclassification rate Got 137 / 140 correct => accuracy: 0.978571

with 7 levels gini Got 130 / 140 correct => accuracy: 0.928571
with 7 levels entropy Got 135 / 140 correct => accuracy: 0.964286
with 7 levels misclassification rate Got 139 / 140 correct => accuracy: 0.992857


 - From above results we can see with increasing the number of levels the accuracy is increasing

### Comparing with sklearn

In [113]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])
predictions_lib = clf.predict(df_input)

In [114]:
print('With library Got %d / %d correct => accuracy: %f' % getAccuracy(predictions_lib,test_df))

With library Got 135 / 140 correct => accuracy: 0.964286


### Output the tree

In [115]:
def dfs(TreeNode,level,f):
    if TreeNode is None:
        return
    if TreeNode.isLeaf:
        print(" "*level,end="")
        print(TreeNode.final_class)
        f.write((" "*level)+str(TreeNode.final_class)+"\n")
    else:
        print(" "*level,end="")
        print("Is ",TreeNode.colToSplitOn," < ",TreeNode.valueToSplitBy)
        f.write(" "*level+"Is "+TreeNode.colToSplitOn+" < "+str(TreeNode.valueToSplitBy)+"\n")
        if hasattr(TreeNode,'left'):
            print(" "*level,end="")
            print("True Branch")
            f.write(" "*level+"True Branch\n")
            dfs(TreeNode.left,level+1,f)
        if hasattr(TreeNode,'right'):
            print(" "*level,end="")
            print("False Branch")
            f.write(" "*level+"False Branch"+"\n")
            dfs(TreeNode.right,level+1,f)

In [151]:
f = open("outputimp_entropy.txt", "w+")
dfs(tree_entropy,0,f)
f.close()

Is  UCShape  <  2.5
True Branch
 Is  CT  <  5.5
 True Branch
  Is  BN  <  4.5
  True Branch
   2
  False Branch
   Is  SECSize  <  1.5
   True Branch
    4
   False Branch
    2
 False Branch
  Is  BN  <  1.5
  True Branch
   2
  False Branch
   4
False Branch
 Is  BN  <  2.5
 True Branch
  Is  UCSize  <  3.5
  True Branch
   Is  NN  <  3.5
   True Branch
    2
   False Branch
    4
  False Branch
   Is  CT  <  8.5
   True Branch
    Is  NN  <  9.5
    True Branch
     Is  SECSize  <  8.5
     True Branch
      Is  MA  <  9.5
      True Branch
       Is  BC  <  3.0
       True Branch
        2
       False Branch
        Is  Mitoses  <  1
        True Branch
         2
      False Branch
       4
     False Branch
      4
    False Branch
     4
   False Branch
    4
 False Branch
  Is  MA  <  5.5
  True Branch
   Is  CT  <  6.5
   True Branch
    Is  BC  <  4.5
    True Branch
     Is  NN  <  7.5
     True Branch
      Is  UCSize  <  6.5
      True Branch
       Is  SECSize  <  6.5
  

### Find correlation and compare

In [129]:
train_df.corr()

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
CT,1.000000,0.663995,0.676188,0.472666,0.517415,0.596813,0.563475,0.539846,0.359995,0.731358
UCSize,0.663995,1.000000,0.906161,0.677183,0.745754,0.690134,0.716310,0.717706,0.478019,0.796081
UCShape,0.676188,0.906161,1.000000,0.650951,0.716182,0.710129,0.703104,0.709958,0.449379,0.806795
MA,0.472666,0.677183,0.650951,1.000000,0.582319,0.661628,0.626146,0.599172,0.419562,0.669506
SECSize,0.517415,0.745754,0.716182,0.582319,1.000000,0.577398,0.595090,0.618441,0.483402,0.663831
BN,0.596813,0.690134,0.710129,0.661628,0.577398,1.000000,0.680660,0.578398,0.346066,0.820500
BC,0.563475,0.716310,0.703104,0.626146,0.595090,0.680660,1.000000,0.650556,0.338910,0.726663
NN,0.539846,0.717706,0.709958,0.599172,0.618441,0.578398,0.650556,1.000000,0.420677,0.700299
Mitoses,0.359995,0.478019,0.449379,0.419562,0.483402,0.346066,0.338910,0.420677,1.000000,0.433060
Diagnosis,0.731358,0.796081,0.806795,0.669506,0.663831,0.820500,0.726663,0.700299,0.433060,1.000000


 - From the above table we can see UCShape and UCSize are highly correlated. Hence dropping UCSize
 - Also dropping Mitoses because the correlation is very less with Diagnosis

In [136]:
train_df_rf = train_df.drop(columns=['UCSize','Mitoses'])
test_df_rf = test_df.drop(columns=['UCSize','Mitoses'])

In [137]:
tree_rf_gini = treeNode(train_df_rf,'gini')
tree_rf_entropy = treeNode(train_df_rf,'entropy')
tree_rf_misclassification_rate = treeNode(train_df_rf,'misclassification_rate')

In [138]:
df_rf_input = test_df_rf.iloc[:,:-1]

In [139]:
y_test_pred_gini_rf = predict(df_rf_input,tree_rf_gini)
y_test_pred_entropy_rf = predict(df_rf_input,tree_rf_entropy)
y_test_pred_mis_rf = predict(df_rf_input,tree_rf_misclassification_rate)

#print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))
print('For reduced features gini Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_gini_rf,test_df_rf))
print('For reduced features entropy Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_entropy_rf,test_df_rf))
print('For reduced features misclassification rate Got %d / %d correct => accuracy: %f' % getAccuracy(y_test_pred_mis_rf,test_df_rf))

For reduced features gini Got 136 / 140 correct => accuracy: 0.971429
For reduced features entropy Got 132 / 140 correct => accuracy: 0.942857
For reduced features misclassification rate Got 135 / 140 correct => accuracy: 0.964286


 - from above values we can see accuracy improved with gini but decreased for misclassification and entropy

### Advantages and Disadvantages of decesion tree

#### Advantages
 - Compared to other algorithms decision trees requires less effort for data preparation during pre-processing.
 - A decision tree does not require normalization of data.
 - A Decision tree model is very intuitive and easy to explain to technical teams as well as stakeholders.
 
#### Disadvantages
 - For a Decision tree sometimes calculation can go far more complex compared to other algorithms.
 - Decision tree often involves higher time to train the model.
 - The Decision Tree algorithm is inadequate for applying regression and predicting continuous values.
 